In [0]:
# Databricks notebook source

"""
# 01_ingest_bronze_autoloader
Autor: Leonardo Lima
Data: 02/01/2026
Descrição:
    Este notebook realiza a ingestão incremental de arquivos JSON brutos para a camada Bronze (Delta Lake).
    Utiliza o Databricks Autolaoder (cloudFiles) para a descoberta eficiente de arquivos.

Arquitetura:
    Raw (JSON) -> [Autolaoder] -> Bronze (Delta)

Features:
    - Schema Evolution: Novos campos são adicionados automaticamente.
    - Rescued Data: Dados corrompidos ou tipos incompatíveis vão para a coluna _rescued_data.
    - Trigger AvailableNow: Executa como batch (custo reduzido) mas com tracking de streaming.
"""
from pyspark.sql.functions import *



In [0]:
# 1. Configuração de Parâmetros (Widgets)
# Em produção, esses valores vêm do Azure Data Factory ou Databricks Workflows

volume_root = "/Volumes/main/default/lakehouse"

dbutils.widgets.text("source_path", f"{volume_root}/raw_customers", "1. Caminho Origem (Raw)")
dbutils.widgets.text("target_path", f"{volume_root}/bronze_customers", "2. Caminho Destino (Bronze)")
dbutils.widgets.text("checkpoint_path", f"{volume_root}/checkpoints/bronze_customers", "3. Checkpoint Path")

source_path = dbutils.widgets.get("source_path")
target_path = dbutils.widgets.get("target_path")
checkpoint_path = dbutils.widgets.get("checkpoint_path")

print(f"--- Configuração do Pipeline ---")
print(f"Origem (Raw):      {source_path}")
print(f"Destino (Bronze):  {target_path}")
print(f"Checkpoint:        {checkpoint_path}")

In [0]:
# 2. Leitura com Autoloader (cloudFiles)
# O Autoloader detecta automaticamente novos arquivos sem precisar listar o diretório inteiro. 

df_stream = (spark.readStream
             .format("cloudFiles")
             .option("cloudFiles.format", "json")
             # inferColumnTypes: Tenta adivinhar se é int, boolean, etc.
             # schemaEvolutionMode: addNewColumns permite que o schema mude sem quebrar o job.
             .option("cloudFiles.inferColumnTypes", "true")
             .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
             
             # rescuedDataColumn: Salva dados que não batem com o schema (evita perda de dados/DLQ)
             .option("cloudFiles.rescuedDataColumn", "_rescued_data")

             # Onde o Autoloader guarda o estado do schema detectado
             .option("cloudFiles.schemaLocation", f"{checkpoint_path}/schema")

             .load(source_path)
             )


In [0]:
# 3. Transformações leves (Enriquecimento de Metadados)
# Na Bronze, mantemos o dado mais original possível, apenas adicionamos rastreabilidade.

df_bronze = df_stream.select(
    "*",
    current_timestamp().alias("ingestion_ts"), # Quando o dado entrou no Lake
    col("_metadata.file_path").alias("source_file") # De qual arquivo veio
)


In [0]:
# 4. Escrita na Camada Bronze (Delta Lake)
# Trigger.AvailableNow é o segredo para rodar Streaming em Jobs agendados (Batch)

(df_bronze.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .outputMode("append") # Bronze sempre append (histórico completo)
 .trigger(availableNow=True) # Processa tudo o que tem pendente e para o cluster
 .start(target_path)
 .awaitTermination()
 )

print("Ingestão Bronze concluída com sucesso.")

### Validação 

In [0]:
%sql
-- Substitua o caminho se o seu for diferente
SELECT * FROM delta.`/Volumes/main/default/lakehouse/bronze_customers`

In [0]:
%sql
-- Validação de Volumetria e Qualidade
SELECT 
  count(*) as total_linhas,
  count(distinct source_file) as total_arquivos_processados,
  min(ingestion_ts) as primeira_ingestao,
  max(ingestion_ts) as ultima_ingestao
FROM delta.`/Volumes/main/default/lakehouse/bronze_customers`

In [0]:
%sql
-- Auditoria de Lineage: Quantos registros vieram de cada arquivo?
SELECT 
  source_file, 
  count(*) as registros_ingeridos,
  min(ingestion_ts) as data_primeira_leitura
FROM delta.`/Volumes/main/default/lakehouse/bronze_customers`
GROUP BY source_file
ORDER BY source_file

In [0]:
%sql
-- Delta Time Travel (Histórico)
DESCRIBE HISTORY delta.`/Volumes/main/default/lakehouse/bronze_customers`